In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q func-timeout

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
###########################                                               37.8%^C


In [ ]:
import pandas as pd
import subprocess
import requests
import json
import time
import re
import os
from func_timeout import func_timeout, FunctionTimedOut

**Khởi động Ollama**

In [ ]:
ollama_process = subprocess.Popen(['ollama', 'serve'],
                                   stdout=subprocess.PIPE,
                                   stderr=subprocess.PIPE)

print("Đang khởi động Ollama server...")
time.sleep(5)

# Kiểm tra server
try:
    response = requests.get('http://localhost:11434')
    print("✓ Ollama server đã sẵn sàng!")
except:
    print("✗ Lỗi khi khởi động server")

**Pull model**

In [ ]:
!ollama pull qwen2.5-coder:14b

In [ ]:
!ollama list

NAME                               ID              SIZE      MODIFIED       
qwen2.5-coder:14b                  9ec8897f747e    9.0 GB    24 minutes ago    
qwen2.5-coder:14b-instruct-q4_0    4463d4082ea3    8.5 GB    32 minutes ago    


**Load data**

In [ ]:
test_path = '/content/drive/MyDrive/T2C_qwen214b_bs/text2cypher_test.csv'
checkpoint_path = '/content/drive/MyDrive/T2C_qwen214b_bs/qwen214b_bs.csv'
test_df = pd.read_csv(test_path, encoding="utf-8-sig")
print(f"✓ Loaded test data shape: {test_df.shape}")

✓ Loaded test data shape: (4833, 6)


**Tạo prompt**

In [ ]:
def prompt(question, schema):
    system_message = """Task: Generate a Cypher statement to query a graph database. Instructions: Use only the provided relationship types and properties in the schema. Do not use any other relationship types or properties that are not provided in the schema. Do not include any explanations or apologies in your responses. Do not respond to any questions that ask anything other than constructing a Cypher statement. Do not include any text except the generated Cypher statement."""

    user_content = f"""Generate Cypher statement to query a graph database. Use only the provided relationship types and properties in the schema.
Schema: {schema}
Question: {question}
Cypher output:"""

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_content}
    ]

    return messages

print("✓ prompt loaded")

✓ prompt loaded


**Tạo cypher**

In [ ]:
def generate_cypher(question, schema, timeout=300):
    messages = prompt(question, schema)

    def _generate():
        # Gọi Ollama API
        response = requests.post(
            'http://localhost:11434/api/chat',
            json={
                "model": "qwen2.5-coder:14b",
                "messages": messages,
                "stream": False,
                "options": {
                    "temperature": 0.1,
                    "top_p": 0.9,
                    "num_predict": 256
                }
            },
            timeout=timeout
        )

        if response.status_code == 200:
            result = response.json()
            content = result['message']['content']

            # Bước 1: Lấy nội dung trong cặp ``` ```
            code_block_match = re.search(r'```(?:cypher)?\s*(.*?)```', content, re.DOTALL | re.IGNORECASE)
            if code_block_match:
                content = code_block_match.group(1).strip()

            # Bước 2: Tìm và lấy từ MATCH trở đi (case-insensitive)
            match_pos = re.search(r'\b(MATCH|match)\b', content)
            if match_pos:
                content = content[match_pos.start():]

            # Bước 3: Xử lý content như code cũ
            content = content.strip().replace('\n', ' ')
            content = re.sub(r'\s+', ' ', content)
            content = content.rstrip(';').strip()

            return content
        else:
            print(f"[ERROR] API returned status {response.status_code}")
            return "error"

    try:
        return func_timeout(timeout, _generate)
    except FunctionTimedOut:
        print(f"[TIMEOUT] Generation exceeded {timeout}s")
        return "time_error"
    except Exception as e:
        print(f"[ERROR] Generation failed: {e}")
        return "error"

print("✓ generate_cypher loaded")

✓ generate_cypher loaded


**Dòng mẫu**

In [ ]:
first_row = test_df.iloc[5]
test_question = first_row['question']
test_schema = first_row['schema']

print("="*80)
print("TEST QUESTION:")
print("="*80)
print(test_question)

# ============================================================================
# Test 1: generate_cypher
# ============================================================================
print("\n" + "="*80)
print("TEST 1: Generated Cypher")
print("="*80)
final_cypher = generate_cypher(test_question, test_schema, timeout=300)
print(final_cypher)

# ============================================================================
# Test 2: EXPECTED CYPHER
# ============================================================================
print("\n" + "="*80)
print("TEST 2: Expected Cypher")
print("="*80)
if 'cypher' in test_df.columns:
    print(first_row['cypher'])
else:
    print("N/A")

TEST QUESTION:
What are the district names and city populations for all districts that between 200,000 and 2,000,000 residents?

TEST 1: Generated Cypher
MATCH (d:District) WHERE d.City_Population >= 200000 AND d.City_Population <= 2000000 RETURN d.District_name, d.City_Population

TEST 2: Expected Cypher
MATCH (n:District) WHERE n.City_Population >= 200000 AND n.City_Population <= 2000000 RETURN n.District_name, n.City_Population


**Reset server mỗi 500 dòng**

In [ ]:
def restart_ollama_server():
    global ollama_process

    print("[RESTART] Đang restart Ollama server...")

    # Kill process cũ
    try:
        ollama_process.terminate()
        ollama_process.wait(timeout=10)
    except:
        ollama_process.kill()

    time.sleep(3)

    # Khởi động lại
    ollama_process = subprocess.Popen(['ollama', 'serve'],
                                       stdout=subprocess.PIPE,
                                       stderr=subprocess.PIPE)
    time.sleep(5)

    print("[RESTART] Ollama server đã khởi động lại!")

In [ ]:
def run_batch(
    test_df,
    checkpoint_path,
    timeout=600,
    log_interval=100,
    restart_interval=500
):
    """
    Chạy batch generation với restart server và tính thời gian chính xác
    """
    # ==========================================================================
    # BƯỚC 1: Kiểm tra và load checkpoint
    if os.path.exists(checkpoint_path):
        print(f"[CHECKPOINT] Tìm thấy file checkpoint: {checkpoint_path}")
        df = pd.read_csv(checkpoint_path, encoding="utf-8-sig")
        print(f"[CHECKPOINT] Đã load {len(df)} dòng từ checkpoint")

        processed_count = df['cypher_generated'].notna().sum()
        print(f"[CHECKPOINT] Đã xử lý: {processed_count}/{len(df)} dòng")

    else:
        print(f"[CHECKPOINT] Không tìm thấy checkpoint, tạo mới từ test_df")
        df = test_df.copy()
        df['cypher_generated'] = ''

        df.to_csv(checkpoint_path, index=False, encoding='utf-8-sig')
        print(f"[CHECKPOINT] Đã tạo file checkpoint: {checkpoint_path}")

    # ==========================================================================
    # BƯỚC 2: Xử lý các dòng chưa có kết quả
    total_rows = len(df)
    batch_start_idx = 0
    processed_since_last_log = 0
    processed_since_last_restart = 0
    actual_processed = 0

    print(f"\n{'='*80}")
    print(f"BẮT ĐẦU XỬ LÝ - Tổng số dòng: {total_rows}")
    print(f"Timeout per query: {timeout}s")
    print(f"Restart mỗi: {restart_interval} dòng")
    print(f"{'='*80}\n")

    restart_ollama_server()
    start_time = time.time()

    for idx in range(total_rows):
        current_cypher = df.at[idx, 'cypher_generated']

        if pd.notna(current_cypher) and str(current_cypher).strip() != '':
            continue

        # ======================================================================
        # XỬ LÝ DÒNG CHƯA CÓ KẾT QUẢ
        actual_processed += 1
        print(f"[Processing] Dòng {idx}...", end=" ", flush=True)

        try:
            question = df.at[idx, 'question']
            schema = df.at[idx, 'schema']

            result = generate_cypher(question, schema, timeout=timeout)

            df.at[idx, 'cypher_generated'] = result

            if result == "error":
                print("ERROR")
            elif result == "time_error":
                print("TIME ERROR")
            else:
                print("SUCCESS")

            processed_since_last_log += 1
            processed_since_last_restart += 1

        except Exception as e:
            print(f"ERROR - {str(e)}")
            df.at[idx, 'cypher_generated'] = "error"
            processed_since_last_log += 1
            processed_since_last_restart += 1

        # ======================================================================
        # RESTART OLLAMA SERVER SAU MỖI restart_interval DÒNG
        # ======================================================================
        if processed_since_last_restart >= restart_interval:
            df.to_csv(checkpoint_path, index=False, encoding='utf-8-sig')
            print(f"[CHECKPOINT] Đã lưu trước khi restart")

            restart_ollama_server()

            processed_since_last_restart = 0
            processed_since_last_log = 0  # Reset cả log counter

        # ======================================================================
        # LOG THỐNG KÊ VÀ LƯU CHECKPOINT
        # ======================================================================
        elif processed_since_last_log >= log_interval:
            df.to_csv(checkpoint_path, index=False, encoding='utf-8-sig')

            elapsed_time = time.time() - start_time

            # Tính thời gian ước tính ĐÚNG
            if actual_processed > 0:
                avg_time_per_row = elapsed_time / actual_processed
                remaining_to_process = ((df['cypher_generated'].isna()) |
                                       (df['cypher_generated'] == '')).sum()
                estimated_time = avg_time_per_row * remaining_to_process
            else:
                avg_time_per_row = 0
                estimated_time = 0

            # Đếm lại từ DataFrame cho batch hiện tại
            batch_df = df.iloc[batch_start_idx:idx+1]
            batch_success = ((batch_df['cypher_generated'].notna()) &
                           (batch_df['cypher_generated'] != 'error') &
                           (batch_df['cypher_generated'] != 'time_error') &
                           (batch_df['cypher_generated'] != '')).sum()
            batch_error = (batch_df['cypher_generated'] == 'error').sum()
            batch_timeout = (batch_df['cypher_generated'] == 'time_error').sum()

            print(f"\n{'='*80}")
            print(f"[LOG] Dòng {batch_start_idx}-{idx}")
            print(f"{'='*80}")
            print(f"Thành công:     {batch_success}")
            print(f"Error:          {batch_error}")
            print(f"Timeout Error:  {batch_timeout}")
            print(f"Tổng xử lý:     {batch_success + batch_error + batch_timeout}")
            print(f"Tiến độ:        {idx + 1}/{total_rows} ({(idx + 1)/total_rows*100:.2f}%)")
            print(f"Thời gian:      {elapsed_time/60:.2f} phút")
            print(f"Ước tính còn:   {estimated_time/60:.2f} phút")
            print(f"[CHECKPOINT] Đã lưu sau {processed_since_last_log} dòng")
            print(f"{'='*80}\n")

            batch_start_idx = idx + 1
            processed_since_last_log = 0

    # ==========================================================================
    # LƯU CHECKPOINT CUỐI CÙNG
    # ==========================================================================
    if processed_since_last_log > 0:
        df.to_csv(checkpoint_path, index=False, encoding='utf-8-sig')
        print(f"[CHECKPOINT] Đã lưu {processed_since_last_log} dòng cuối cùng")

    # ==========================================================================
    # KẾT THÚC - LOG CUỐI CÙNG
    # ==========================================================================
    total_time = time.time() - start_time

    final_success = (df['cypher_generated'].notna() &
                     (df['cypher_generated'] != 'error') &
                     (df['cypher_generated'] != 'time_error') &
                     (df['cypher_generated'] != '')).sum()
    final_error = (df['cypher_generated'] == 'error').sum()
    final_timeout = (df['cypher_generated'] == 'time_error').sum()

    print(f"\n{'='*80}")
    print(f"HOÀN THÀNH")
    print(f"{'='*80}")
    print(f"Tổng số dòng:        {total_rows}")
    print(f"Thành công:          {final_success} ({final_success/total_rows*100:.2f}%)")
    print(f"Error:               {final_error} ({final_error/total_rows*100:.2f}%)")
    print(f"Timeout Error:       {final_timeout} ({final_timeout/total_rows*100:.2f}%)")
    print(f"Tổng thời gian:      {total_time/60:.2f} phút")
    print(f"{'='*80}")

    return df

print("✓ run_batch loaded")

✓ run_batch loaded


In [ ]:
result_df = run_batch(
    test_df=test_df,
    checkpoint_path=checkpoint_path
)

[CHECKPOINT] Không tìm thấy checkpoint, tạo mới từ test_df
[CHECKPOINT] Đã tạo file checkpoint: /content/drive/MyDrive/T2C_qwen214b_bs/qwen214b_bs.csv

BẮT ĐẦU XỬ LÝ - Tổng số dòng: 4833
Timeout per query: 600s
Restart mỗi: 500 dòng

[RESTART] Đang restart Ollama server...
[RESTART] Ollama server đã khởi động lại!
[Processing] Dòng 0... SUCCESS
[Processing] Dòng 1... SUCCESS
[Processing] Dòng 2... SUCCESS
[Processing] Dòng 3... SUCCESS
[Processing] Dòng 4... SUCCESS
[Processing] Dòng 5... SUCCESS
[Processing] Dòng 6... SUCCESS
[Processing] Dòng 7... SUCCESS
[Processing] Dòng 8... SUCCESS
[Processing] Dòng 9... 

KeyboardInterrupt: 